In [185]:
import pandas as pd
import numpy as np
import re
# !pip install openpyxl

In [186]:
df_all = pd.read_excel(r"C:\Users\Jinhyun\Documents\GitHub\Python\Investment\Top stock pick\2022-02\Feb stock pick data.xlsx", sheet_name=None, header = 1)

In [187]:
def counting_stockpicking(df_all, min_count, save):
    report_lst = list(df_all.keys())
    emp_lst = []

    for report in report_lst:
        check = True
        report_cols = list(df_all[report].columns)
        for col in report_cols:
            if col in ['SYMBOL','Symbol','TICKER', 'Ticker', 'Stock (ticker)', 'Company / Ticker','Stocks', 'Company', 'Stock']:
                if '(' in str(list(df_all[report][col])[0]):
                    tmp_lst = list(map(lambda x : re.findall('\(([^)]+)', x), list(df_all[report][col])))
                    emp_lst.append(sum(tmp_lst, []))
                else:
                    emp_lst.append(list(df_all[report][col]))
                check = False

        if check:
            print("*************************************")
            print("Warning!!")
            print("No ticker or symbol")
            print("Need an addittional action")
            print("Report name: ", report)
            print("Report columns: ", df_all[report].columns)
            print("*************************************")
    print('*************************************')
    print('Counted TICKERS')
    result = sum(emp_lst,[])
    count_result =[(i, result.count(i)) for i in result if result.count(i) >= min_count]
    result_rank = pd.DataFrame(sorted(list(set(count_result)), key=lambda x : x[1], reverse = True), columns = ['TICKER','COUNT'])
    # print(result)

    if save.upper() == 'YES':
        result_rank.to_excel('result_rank.xlsx')

    return result_rank

In [188]:
def Theme_categorizing(df_all, min_count, save):
    report_lst = list(df_all.keys())

    filter_lst = []
    result_dic = {}
    for report in report_lst:
        emp_lst = []
        check = True
        report_cols = list(df_all[report].columns)
        for col in report_cols:
            if col in ['SYMBOL','Symbol','TICKER', 'Ticker', 'Stock (ticker)', 'Company / Ticker']:
                if '(' in str(list(df_all[report][col])[0]):
                    emp_lst.append(sum(list(map(lambda x : re.findall('\(([^)]+)', x), list(df_all[report][col]))),[]))
                else:
                    emp_lst.append(list(df_all[report][col]))
                result_dic[report] = emp_lst
                check = False            
    result = sum(emp_lst,[])
    # print(result)
    # print(result_dic)
    if save.upper() == 'YES':
        pd.DataFrame(result_dic).stack().to_excel(f'Theme_minimum_count_{min_count}.xlsx')
    return result_dic



In [189]:
def count_theme(count_xlxs, theme_xlxs):
    for i in theme_xlxs.keys():
        ticker_checker = sum(theme_xlxs[i], [])
        count_xlxs[f'{i}'] = count_xlxs['TICKER'].apply(lambda x: i if x in ticker_checker else ',')
    count_xlxs['Theme'] = count_xlxs[theme_xlxs.keys()].apply(' '.join, axis=1).apply(lambda x: re.split(",", x)).apply(lambda x: list(set(x))).apply(lambda x: sorted(x))
    count_xlxs['Count'] = count_xlxs['Theme'].apply(lambda x: len(x)- 2)
    count_xlxs = count_xlxs.drop(list(theme_xlxs.keys()), axis = 'columns')
    count_xlxs = count_xlxs.drop('COUNT', axis = 'columns')
    count_xlxs = count_xlxs.sort_values(by = 'Count', ascending = False)

    count_xlxs.to_excel(f'count_theme.xlsx')
    
    return count_xlxs    
    # count_xlxs.to_excel(f'count_theme.xlsx')


In [190]:
count_theme(counting_stockpicking(df_all, 3,'yes'), Theme_categorizing(df_all, 0, 'YES'))

*************************************
Counted TICKERS


,TICKER,Theme,Count
0,MU,"[, , 20 Tech Stocks with safety net , BMO’...",9
1,COP,"[, , 2016 FED RATE HIKE WINNERS , An earnin...",8
2,QCOM,"[, , An earnings-season screen of wi , Buy-...",7
3,XOM,"[, , Barclays Overweight List with l , BofA...",7
6,PYPL,"[, , Inflation Sensitive Sectors _ S , NASD...",7
...,...,...,...
90,JNJ,"[, , STOCKS WITH HIGH AND STABLE GRO , Sect...",2
121,Synopsys Inc.,"[, ]",0
115,News Corp.,"[, ]",0
84,KLA Corp.,"[, ]",0
